In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import glob
import json

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/binance','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Production Config':
                    d['prod'] = True if value == 'True' else False

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip())/24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())


        big_data = big_data.append(d, ignore_index=True)

# files = glob.iglob('backtests/binance_production/' + '**/backtest_result.txt', recursive=True)
# i = 0
# for filename in files:
#     #print(f'parsing : {filename}')
#     json_file = filename.replace('backtest_result.txt','live_config.json')
#     with open(json_file) as f:
#         live_config = json.load(f)
#         config_type = live_config['config_name']
#
#     with open(filename, 'r') as file:
#         data = file.read().split('\n')
#         d = {'file': filename.replace('backtests/binance_production','')}
#         current_state = 'long'
#         for l in data:
#             l = l.split('|')
#             if len(l) == 4:
#                 key = l[1].strip()
#                 value = l[2].strip()
#
#                 if key == 'Passivbot Version':
#                     d['v'] = value
#                 if key == 'Symbol':
#                     d['s'] = value
#
#                 if key == 'Long':
#                     current_state = 'long'
#                     d['long'] = bool(value) or False
#                 if key == 'Short':
#                     current_state = 'short'
#                     d['short'] = bool(value) or False
#
#                 if key == 'Total gain' and current_state == 'long':
#                     d['tgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'long':
#                     d['adgl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'long':
#                     d['bnkrl_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'long':
#                     d['ebrl'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'long':
#                     d['maxsl'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'long':
#                     d['pamaxl'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'long':
#                     d['pastdl'] = float(value.strip())
#
#
#                 if key == 'Total gain' and current_state == 'short':
#                     d['tgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Average daily gain' and current_state == 'short':
#                     d['adgs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Closest bankruptcy' and current_state == 'short':
#                     d['bnkrs_%'] = float(value.replace('%', '').strip())
#                 if key == 'Lowest equity/balance ratio' and current_state == 'short':
#                     d['ebrs'] = float(value.replace('%', '').strip())
#                 if key == 'Max hours stuck' and current_state == 'short':
#                     d['maxss'] = float(value.replace('%', '').strip())/24
#                 if key == 'Price action distance max' and current_state == 'short':
#                     d['pamaxs'] = float(value.replace('%', '').strip())
#                 if key == 'Price action distance std' and current_state == 'short':
#                     d['pastds'] = float(value.strip())
#
#         d['prod'] = True
#         big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_609468/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_609468/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_609468/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_609468/3023065081.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod
0,\1000SHIBUSDT\plots\2022-04-03T194816\backtest...,5.5,1000SHIBUSDT,True,52.53,0.1780,0.066776,0.644511,100.0,0.8613,12.442375,True,-96.020,-1.88000,0.146280,0.802360,5.447,0.10870,155.190833,NaN
1,\1000SHIBUSDT\plots\2022-04-03T204145\backtest...,5.5,1000SHIBUSDT,True,26.73,0.0850,0.070106,0.830417,100.0,0.9153,6.572917,True,-95.820,-1.88000,0.148225,0.803949,5.404,0.09669,155.191667,NaN
2,\1000SHIBUSDT\plots\2022-04-06T180851\backtest...,5.5,1000SHIBUSDT,True,90.63,0.2460,0.065637,0.629854,100.0,0.7673,12.442375,True,-91.820,-1.64000,0.124514,0.747041,5.959,0.08869,180.079167,True
3,\1000XECUSDT\plots\2022-04-06T155612\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
4,\1000XECUSDT\plots\2022-04-06T181135\backtest_...,5.5,1000XECUSDT,True,16.60,0.0401,0.056577,0.843518,100.0,0.8579,5.065292,True,49.790,0.20700,0.062401,0.371501,100.000,0.86990,21.804875,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,\ZENUSDT\plots\2022-04-08T000448\backtest_resu...,5.5,ZENUSDT,True,78.47,0.2110,0.129356,0.981507,100.0,0.8117,14.342375,True,8.361,0.19300,0.132544,0.571631,19.640,0.25670,47.980000,False
379,\ZILUSDT\plots\2022-04-07T124653\backtest_resu...,5.5,ZILUSDT,True,11.86,0.0150,0.213304,1.171540,100.0,0.8399,35.809042,True,-90.810,-1.64000,0.128495,0.776683,5.828,0.07862,33.732625,True
380,\ZILUSDT\plots\2022-04-13T093427\backtest_resu...,5.5,ZILUSDT,True,24.10,0.0730,0.077813,0.534668,100.0,0.7921,13.360417,True,-90.650,-1.64000,0.125380,0.780698,5.950,0.07147,33.434708,False
381,\ZRXUSDT\plots\2022-04-07T125157\backtest_resu...,5.5,ZRXUSDT,True,57.37,0.1690,0.114260,0.708101,100.0,0.8835,22.423625,True,20.130,0.00183,0.141192,0.532109,93.390,0.63730,47.927917,True


Now filter the dataframe

In [4]:
# allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
#                    "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
#                    "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
#                    "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
#                    "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
#                    "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
#                    "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
#                    "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
#                    "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
#                    "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
#                    "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
#                    "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
#                    "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
#                    "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
#                    "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
#                    "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
#                    "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
#                    "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
#                    "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
#                    "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = [
    #"OMG","DUSK","GTC"
    # "LRC",
    # "--------------------------",
    # "SNX", "RSR",
    # "--------------------------",
    # "CELO", "ROSE", "OCEAN","SXP","DENT"
    # "SFP","ENS","DODO","AKRO","REEF","NEO",
    # "----------------------------------------",
    # "1000XEC","AKRO","ALPHA","ANKR","BAKE","BAND","BEL","BTS","C98","COMP",
    "-----------------------------------",
    # "DGB","DOGO","FLM","FLOW","HBAR",
    # "-----------------------------------",
    # "HOT","ICX","IOST","IOTA","KAVA","KLAY","LINA","LIT","MKR","MTL","NEO","NKN","OGN","ONT"
    #"QTUM","REEF","RLC","RVN","SC","SFP","STMX","TOMO","TRB","UNFI","WAVES", "ZIL", "ZRX"
    #"COTI"
    #"RUNE","UNI"
    #"BAT", "BAL", "SRM", "RUNE",
    "BAT", "BAL", "SRM", "RUNE" , 'STORJ','UNI','FTM','FIL','LRC','CVC',"CTK","SKL","SAND","XEM"
]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
    & (big_data['long'] == True)
    & (big_data['adgl_%'] > 0.04)
    #& (big_data['short'] == True)
    #& (big_data['adgt_%'] > 0.1)
    & (big_data['bnkrl_%'] > 95.0)
    #& (big_data['bnkrs_%'] > 95.0)
    #& (big_data['ebrs'] > .85)
    & (big_data['ebrl'] > .8)
    & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,...,short,tgs_%,adgs_%,pastds,pamaxs,bnkrs_%,ebrs,maxss,prod,adgt_%
41,\BALUSDT\plots\2022-04-16T001557\backtest_resu...,5.5,BALUSDT,True,31.86,0.0753,0.117695,0.891543,100.0,0.8547,...,True,19.490,0.0706,0.114573,0.516838,100.000,0.72590,29.197917,False,0.1459
45,\BATUSDT\plots\2022-04-05T233736\backtest_resu...,5.5,BATUSDT,True,26.75,0.0989,0.066609,0.491816,100.0,0.8877,...,True,3.470,0.2920,0.160006,0.692660,8.508,0.11740,147.241667,NaN,0.3909
46,\BATUSDT\plots\2022-04-06T004844\backtest_resu...,5.5,BATUSDT,True,23.21,0.0780,0.090021,0.771371,100.0,0.9273,...,True,8.286,0.0324,0.137536,0.642917,100.000,0.65950,36.501375,NaN,0.1104
47,\BATUSDT\plots\2022-04-15T233053\backtest_resu...,5.5,BATUSDT,True,43.96,0.1220,0.055084,0.340868,100.0,0.8646,...,True,14.170,0.0829,0.122019,0.486527,99.510,0.67480,43.581250,False,0.2049
51,\BATUSDT\plots\2022-04-16T002527\backtest_resu...,5.5,BATUSDT,True,44.90,0.1110,0.092296,0.774568,100.0,0.8930,...,True,7.711,0.0880,0.149616,0.642637,46.580,0.42580,47.924167,False,0.1990
91,\CTKUSDT\plots\2022-04-16T142623\backtest_resu...,5.5,CTKUSDT,True,79.73,0.2140,0.100915,1.107460,100.0,0.8742,...,True,12.880,0.1100,0.126736,0.665903,39.310,0.37710,40.442375,False,0.3240
94,\CVCUSDT\plots\2022-04-16T105816\backtest_resu...,5.5,CVCUSDT,True,49.99,0.1420,0.149732,1.017340,100.0,0.8730,...,True,1.880,0.4230,0.116818,0.676763,18.720,0.25420,40.179167,False,0.5650
141,\FTMUSDT\plots\2022-04-03T084031\backtest_resu...,5.5,FTMUSDT,True,89.78,0.2730,0.132510,1.193860,100.0,0.8551,...,True,-54.870,0.2760,0.216012,0.772471,7.450,0.09939,32.865292,NaN,0.5490
142,\FTMUSDT\plots\2022-04-03T090001\backtest_resu...,5.5,FTMUSDT,True,62.81,0.2030,0.169560,1.267070,100.0,0.9255,...,True,-93.210,-1.8800,0.204906,0.818578,5.961,0.07093,206.972083,NaN,-1.6770
151,\FTMUSDT\plots\2022-04-16T105445\backtest_resu...,5.5,FTMUSDT,True,87.22,0.2060,0.173026,1.229000,100.0,0.8631,...,True,-93.960,-1.5900,0.179580,0.644577,5.924,0.10890,219.965417,False,-1.3840


In [5]:
df.drop_duplicates(
    subset=['s', 'prod', 'tgl_%', 'tgs_%', 'adgl_%', 'adgs_%', 'adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long',
            'short'], keep='first',
    inplace=True)
dx = df[['file', 'prod', 's', 'adgt_%', 'adgl_%', 'adgs_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'maxsl', 'pamaxl','pastdl',
         'maxss', 'pamaxs','pastds' ]]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_609468/2052413415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(


,file,prod,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs,maxsl,pamaxl,pastdl,maxss,pamaxs,pastds
41,\BALUSDT\plots\2022-04-16T001557\backtest_resu...,False,BALUSDT,0.1459,0.0753,0.0706,100.0,100.000,0.8547,0.72590,21.903458,0.891543,0.117695,29.197917,0.516838,0.114573
45,\BATUSDT\plots\2022-04-05T233736\backtest_resu...,NaN,BATUSDT,0.3909,0.0989,0.2920,100.0,8.508,0.8877,0.11740,8.890958,0.491816,0.066609,147.241667,0.692660,0.160006
46,\BATUSDT\plots\2022-04-06T004844\backtest_resu...,NaN,BATUSDT,0.1104,0.0780,0.0324,100.0,100.000,0.9273,0.65950,10.934708,0.771371,0.090021,36.501375,0.642917,0.137536
47,\BATUSDT\plots\2022-04-15T233053\backtest_resu...,False,BATUSDT,0.2049,0.1220,0.0829,100.0,99.510,0.8646,0.67480,27.697208,0.340868,0.055084,43.581250,0.486527,0.122019
51,\BATUSDT\plots\2022-04-16T002527\backtest_resu...,False,BATUSDT,0.1990,0.1110,0.0880,100.0,46.580,0.8930,0.42580,10.934708,0.774568,0.092296,47.924167,0.642637,0.149616
91,\CTKUSDT\plots\2022-04-16T142623\backtest_resu...,False,CTKUSDT,0.3240,0.2140,0.1100,100.0,39.310,0.8742,0.37710,13.372917,1.107460,0.100915,40.442375,0.665903,0.126736
94,\CVCUSDT\plots\2022-04-16T105816\backtest_resu...,False,CVCUSDT,0.5650,0.1420,0.4230,100.0,18.720,0.8730,0.25420,23.047208,1.017340,0.149732,40.179167,0.676763,0.116818
141,\FTMUSDT\plots\2022-04-03T084031\backtest_resu...,NaN,FTMUSDT,0.5490,0.2730,0.2760,100.0,7.450,0.8551,0.09939,14.206250,1.193860,0.132510,32.865292,0.772471,0.216012
142,\FTMUSDT\plots\2022-04-03T090001\backtest_resu...,NaN,FTMUSDT,-1.6770,0.2030,-1.8800,100.0,5.961,0.9255,0.07093,17.521542,1.267070,0.169560,206.972083,0.818578,0.204906
151,\FTMUSDT\plots\2022-04-16T105445\backtest_resu...,False,FTMUSDT,-1.3840,0.2060,-1.5900,100.0,5.924,0.8631,0.10890,18.450708,1.229000,0.173026,219.965417,0.644577,0.179580
